In [4]:
from asammdf import MDF
import easygui
import pandas as pd
import glob, os
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import binned_statistic_2d 

def Pressure_df(x):
    RPM = [3400,3200,3000,2800,2600,2400,2200,2000,1800,1600,1400,1200]
    mean_pres = []
    for i in RPM:
        df_i = x[(x['cps_n_engine'] > i-50) & (x['cps_n_engine'] < i+50)]
        a = df_i['egr_P_exhaustp'].mean()
        mean_pres.append(a)
    pres_dict = {'Engine Speed':RPM,'Mean Pressure':mean_pres}
    df_mean_pres = pd.DataFrame(pres_dict)
    df_mean_pres.set_index(df_mean_pres['Engine Speed'])
    graph_ = df_mean_pres.plot(x='Engine Speed',y='Mean Pressure',xticks= RPM,kind = 'line',grid=True)

datFiles = easygui.fileopenbox(msg='Select dat files', default='*.dat', multiple = True)
#get the folderpath of dat file
folderPath = os.path.dirname(datFiles[0])

#def read_df():
dfs = {}
for datFile in datFiles:
    mdf = MDF(datFile)
    allSignals = list(mdf.channels_db.keys())
    measuredSignals = []
    #remove signals with CPP in it
    for signal in allSignals:
        if not(("CCP" in signal) or ("_sword" in signal) or "$" in signal):
            measuredSignals.append(signal)
        #creating an empty list of important signals
    impSignals = ['cps_n_engine', 'egr_b_operate_valve', 
                     'egr_T_exhaust_temperature', 'egr_T_oil_temperature',
                     'egr_T_limiting_temp_low', 'egr_T_limiting_temp_high',
                     'egr_P_exhaustp']
    df = mdf.to_dataframe(
            channels=impSignals,
            raster= 'egr_b_operate_valve',
            time_from_zero=True,
            empty_channels="zeros",
    #              keep_arrays=False,
            use_display_names=True,
            time_as_date=True,
            reduce_memory_usage=True,
            raw=False,
            ignore_value2text_conversions=False)
    #remove \CCP from column names of dataframe
    df.columns = [col.split('\\')[0] for col in df.columns]
    #resampling the data to 1 second
    df = df.resample('S').mean()
        
    n_i = list(range(1,len(df['egr_b_operate_valve'])+1))
    df['Sl.no'] = n_i
    df.set_index('Sl.no',inplace=True)
    
    print(df.egr_P_exhaustp.values)
    
"""   
    #Pressure_df(df)
    plt.grid(linestyle='dotted')
    plt.xlabel('RPM')
    plt.ylabel('mbar')
    plt.title('Exhaust Back Pressure')
    plt.show()
    mdf.close()
"""
"""    
def graph(x,a,b)
    graph_ = x.plot(x='a',y='b',xticks= RPM,kind = 'line',grid=True)
    plt.grid(linestyle='dotted')
    plt.xlabel('RPM')
    plt.ylabel('mbar')
    plt.title('Exhaust Back Pressure')
"""   


Sl.no
1      115.034065
2      114.544998
3      114.154549
4      114.762001
5      113.762001
          ...    
356     35.748001
357     30.118999
358     30.000000
359     30.000000
360     30.000000
Name: egr_P_exhaustp, Length: 360, dtype: float32


"    \ndef graph(x,a,b)\n    graph_ = x.plot(x='a',y='b',xticks= RPM,kind = 'line',grid=True)\n    plt.grid(linestyle='dotted')\n    plt.xlabel('RPM')\n    plt.ylabel('mbar')\n    plt.title('Exhaust Back Pressure')\n"